In [1]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import applications
from keras.layers import GlobalAveragePooling2D
from keras.models import model_from_json

D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarnin

In [2]:
import collections
import os
import cv2
import pandas as pd
import numpy as np
import random

In [73]:
NOT_ROTATE = -1
NUMBER_CLASSES = 4

def correct_image(image, rot_correction):
    if rot_correction == NOT_ROTATE:
        return image
    else:
        return cv2.rotate(image, rot_correction)    
    
def preprocess(X, Y):
    """X: list of images
       Y: simple list of numbers"""
    #X =X.astype('float32')
    #X /= 255
    Y = np.array(Y)
    Y = Y.reshape(len(Y), 1)
    Y = keras.utils.to_categorical(Y, NUMBER_CLASSES)
    return X, Y


def rot_corrections(label_code):
    if label_code == 0:
        return cv2.ROTATE_90_CLOCKWISE
    elif label_code == 1:
        return cv2.ROTATE_90_COUNTERCLOCKWISE
    elif label_code == 2:
        return NOT_ROTATE
    elif label_code == 3:
        return cv2.ROTATE_180

def labels_toonehot(label):
    if label == 'rotated_left':
        return 0
    elif label == 'rotated_right':
        return 1
    elif label == 'upright':
        return 2
    elif label == 'upside_down':
        return 3
#[1., 0., 0., 0.] - 'rotated_left'
#[0., 1., 0., 0.] - 'rotated_right'
#[0., 0., 1., 0.] - 'upright'
#[0., 0., 0., 1.] - 'upside_down'
#['rotated_left', 'rotated_right', 'upright', 'upside_down']    



def prediction(file_list):
    data_xi = load_data(file_list)
    ypred_i = loaded_model.predict(data_xi)
    ypred_i = np.argmax(ypred_i, axis=1)
    return ypred_i
    

def load_data(file_list, to_preprocess = True):
    DATA_FOLDER = 'train'
    data = []
    for file in file_list:
        file_path = os.path.join(DATA_FOLDER, file)
        image = cv2.imread(file_path)
        
        if to_preprocess:
            image = preprocess(image)
        data.append(image)

    data = np.array(data)
    return data

def preprocess(image):
    image = cv2.resize(image, (256, 256), interpolation = cv2.INTER_AREA)
    image = image.astype('float32')
    image /= 255
    return image
    
    
    

In [4]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'rotvision_trained_model.h5'

# load json and create model
model_json_path = os.path.join(save_dir, model_name.split('.')[0] + '.json')
model_path = os.path.join(save_dir, model_name)

json_file = open(model_json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_path)

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])    

print("Loaded model from disk")

Loaded model from disk


In [ ]:
# Gerar a validacao aqui

In [5]:
labels = pd.read_csv('train.truth.csv', sep=',')

In [7]:
validation_files = labels.fn.tolist()[34227:]
validation_labels = labels.label.tolist()[34227:]

In [14]:
val_batchi = load_data(validation_files[:32])

In [16]:
ypred = loaded_model.predict(val_batchi)

In [19]:
pred1 = np.argmax(ypred,axis=1)
truth1 = validation_labels[:32]
truth1 = [labels_toonehot(x) for x in validation_labels[:32]]

In [21]:
acertos = []
for pred, truth in zip(pred1, truth1):
    if pred == truth:
        acertos.append(1)
    else:
        acertos.append(0)
np.mean(acertos)

0.90625

In [36]:
batch_size=32
number_batches = int(len(validation_files) / batch_size) + 1
for i in range(number_batches):
    start = i* batch_size
    end = (i + 1) * batch_size
    files_i = validation_files[start:end]
    data_xi = load_data(files_i)
    ypred_i = loaded_model.predict(data_xi)
    ypred_i = np.argmax(ypred_i,axis=1)
    labels_i = [labels_toonehot(x) for x in validation_labels[start:end]]

    for truth, pred in zip(labels_i, ypred_i):
        if pred == truth:
            acertos.append(1)
        else:
            acertos.append(0)


In [50]:
labels_i

[1, 1, 1, 3, 3, 1, 2, 1, 2, 0, 0, 0, 1]

In [74]:
original_images = load_data(validation_files[:32], False)
labels_i = prediction(validation_files[:32])
corrections = [rot_corrections(x) for x in labels_i]
for i in range(32):
    image = original_images[i]
    cv2.imwrite(str(i) + '.jpg', image)
    image_corr = correct_image(image, corrections[i])
    cv2.imwrite(str(i) + '-corr.jpg', image_corr)
    